# 一、summary() 打印网络参数层
# 二、模型参数大小

In [1]:
import torch
from torchsummary import summary
import torch.nn as nn 

from nets.mobilenetv3_torchvision_NYL import mobilenet_v3_large
from nets.yolo import YoloBody


In [2]:
from thop import profile
from copy import deepcopy
import loguru
from loguru import logger 

def get_model_info(model, tsize):

    stride = 64
    img = torch.zeros((1, 3, stride, stride), device=next(model.parameters()).device)
    flops, params = profile(deepcopy(model), inputs=(img,), verbose=False)
    params /= 1e6
    flops /= 1e9
    flops *= tsize[0] * tsize[1] / stride / stride * 2  # Gflops
    info = "Params: {:.2f}M, Gflops: {:.2f}".format(params, flops)
    return info

In [3]:
# 模型参数大小
exp_test_size = [640,640]   # 图像的输入input_shape

# 需要使用device来指定网络在GPU还是CPU运行
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')       

print('YOLOX_Body')
features= YoloBody(num_classes =1, phi='l') 
model = features.to(device) 

logger.info("Model Summary: {}".format(get_model_info(model, exp_test_size))) 

YOLOX_Body


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2021-11-25 14:25:40.536 | INFO     | __main__:<module>:11 - Model Summary: Params: 54.15M, Gflops: 155.31


In [4]:
if __name__ == "__main__":
    
    # 需要使用device来指定网络在GPU还是CPU运行
    device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')       

    print('YOLOX_Body')
    features= YoloBody(num_classes =1, phi='s') 
    model = features.to(device) 
    summary(features, input_size=(3,224,224), batch_size=1, device="cuda")  

YOLOX_Body
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 32, 112, 112]           3,456
       BatchNorm2d-2          [1, 32, 112, 112]              64
              SiLU-3          [1, 32, 112, 112]               0
          BaseConv-4          [1, 32, 112, 112]               0
             Focus-5          [1, 32, 112, 112]               0
            Conv2d-6            [1, 64, 56, 56]          18,432
       BatchNorm2d-7            [1, 64, 56, 56]             128
              SiLU-8            [1, 64, 56, 56]               0
          BaseConv-9            [1, 64, 56, 56]               0
           Conv2d-10            [1, 32, 56, 56]           2,048
      BatchNorm2d-11            [1, 32, 56, 56]              64
             SiLU-12            [1, 32, 56, 56]               0
         BaseConv-13            [1, 32, 56, 56]               0
           Conv2d-14        

In [8]:
# YOLOX每层代码理解与计算
# Flatten不属于层
# ##迭代层会丢掉Flatten()
model =YoloBody(num_classes =2, phi='x')    #创建实例对象 
X = torch.rand(1, 3, 640, 640)
for name, blk in model.named_children(): 
    X = blk(X)
    if isinstance(blk, nn.AdaptiveAvgPool2d): 
        X=torch.flatten(X, 1) 
    print( 'Layer_name=',name, 'Layer_block=', blk) 

Layer_name= backbone Layer_block= YOLOPAFPN(
  (backbone): CSPDarknet(
    (stem): Focus(
      (conv): BaseConv(
        (conv): Conv2d(12, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
    )
    (dark2): Sequential(
      (0): BaseConv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (1): CSPLayer(
        (conv1): BaseConv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (conv2): BaseConv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(8

In [6]:
# 打印网络print(X)
print(model) # 打印model,输出每层网络

YoloBody(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): SiLU()
          )
          (conv2): BaseConv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [7]:
# YOLOX 网络输入图像shape=W,H == 32的倍数
imageshape = []
for i in range(21):
   imageshape.append(32*i)
print(imageshape)


[0, 32, 64, 96, 128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512, 544, 576, 608, 640]
